<a href="https://colab.research.google.com/github/DmitryKutsev/congrats_generator/blob/feature%233/my_gpt3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://colab.research.google.com/github/sberbank-ai/ru-gpts/blob/master/examples/Finetune_RuGPTs_with_HF.ipynb#scrollTo=aZ-4Kav28cH0




# **IMPORT**

In [1]:
import pandas as pd

In [2]:
!pip install transformers -U

In [3]:
!pip install sentencepiece

In [4]:
!pip install tqdm boto3 requests regex sacremoses

In [5]:
import torch
from transformers import AutoModel, AutoTokenizer, AutoConfig

In [6]:
# model = torch.hub.load('huggingface/transformers', 'modelForCausalLM', 'gpt2')   
#  # Download model and configuration from huggingface.co and cache.
# # model = torch.hub.load('huggingface/transformers', 'modelForCausalLM', './test/saved_model/')  
# # E.g. model was saved using `save_pretrained('./test/saved_model/')`
# model = torch.hub.load('huggingface/transformers', 'modelForCausalLM', 'gpt2', output_attentions=True)  
# # Update configuration during loading
# assert model.config.output_attentions == True
# # Loading from a TF checkpoint file instead of a PyTorch model (slower)
# config = AutoConfig.from_pretrained('bert-base-uncased')
# # AutoConfig.from_pretrained('bert-base-uncased')
# # model = torch.hub.load('huggingface/transformers', 'modelForCausalLM', 
# #                        './tf_model/gpt_tf_checkpoint.ckpt.index', from_tf=True, config=config)

In [7]:
!git clone https://github.com/huggingface/transformers.git

fatal: destination path 'transformers' already exists and is not an empty directory.


In [8]:
!pip install tensorboard
# !pip install wandb; wandb login

In [9]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [10]:
!pip install awscli
!aws s3 sync --no-sign-request s3://models.dobro.ai/gpt2/ru/unfreeze_all gpt2

# **PREPARE DATA**

In [11]:
!wget https://raw.githubusercontent.com/DmitryKutsev/congrats_generator/feature%233/crawler/curr_congrats.txt

--2021-12-21 15:10:04--  https://raw.githubusercontent.com/DmitryKutsev/congrats_generator/feature%233/crawler/curr_congrats.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1471875 (1.4M) [text/plain]
Saving to: ‘curr_congrats.txt.5’

curr_congrats.txt.5 100%[===================>]   1.40M  --.-KB/s    in 0.06s   

2021-12-21 15:10:05 (25.4 MB/s) - ‘curr_congrats.txt.5’ saved [1471875/1471875]



In [12]:
with open('curr_congrats.txt') as handler:
  full_text = handler.read()
full_text[:3000]

'\nTITLE\nЭдварду Радзинскому, писателю, драматургу, сценаристу\nCONTENT\n Уважаемый Эдвард Станиславович!Примите поздравления с\xa085-летием.Человек многогранного дарования и\xa0большой созидательной энергии, Вы по\xa0праву входите в\xa0число признанных\nдраматургов, мастеров современного театрального искусства. И\xa0конечно, Вас\nхорошо знают и\xa0любят как автора целого ряда просветительских проектов, чьё\nтворчество завоевало сердца самой широкой аудитории, в\xa0том числе молодёжи.Желаю доброго здоровья, благополучия и\xa0бодрости\nдуха. Владимир Путин     Статус материала Опубликован в\xa0разделе: Телеграммы  Ссылка на материал: kremlin.ru/d/66751   Текстовая версия      \nEND\n\nTITLE\nКоллективу радиостанции «Орфей»\nCONTENT\n Дорогие друзья!Поздравляю вас со\xa0знаменательным юбилеем\xa0– 30-летием радиостанции «Орфей».Все эти годы ваш коллектив объединяет настоящих единомышленников, профессионалов, преданных высокому искусству, стремящихся внести свой вклад в\xa0сбережение луч

In [13]:
full_text.split('END')[:3][0].replace('\n', '').split('CONTENT')

['TITLEЭдварду Радзинскому, писателю, драматургу, сценаристу',
 ' Уважаемый Эдвард Станиславович!Примите поздравления с\xa085-летием.Человек многогранного дарования и\xa0большой созидательной энергии, Вы по\xa0праву входите в\xa0число признанныхдраматургов, мастеров современного театрального искусства. И\xa0конечно, Васхорошо знают и\xa0любят как автора целого ряда просветительских проектов, чьётворчество завоевало сердца самой широкой аудитории, в\xa0том числе молодёжи.Желаю доброго здоровья, благополучия и\xa0бодростидуха. Владимир Путин     Статус материала Опубликован в\xa0разделе: Телеграммы  Ссылка на материал: kremlin.ru/d/66751   Текстовая версия      ']

In [14]:
titles_list = []
congrats_list = []

for letter in full_text.split('END'):
  letter = letter.replace('\n', '').replace('TITLE', '')
  # print(letter)
  # print(letter.split('CONTENT'))
  # print(letter.split('CONTENT'))
  # break
  if len(letter.split('CONTENT')) > 1:
    titles_list.append(letter.split('CONTENT')[0])
    congrats_list.append(letter.split('CONTENT')[1])

print(titles_list[:3])
print(congrats_list[:3])
print(len(titles_list))
print(len(congrats_list))

['Эдварду Радзинскому, писателю, драматургу, сценаристу', 'Коллективу радиостанции «Орфей»', 'Михаилу Ковальчуку, президенту НИЦ «Курчатовский институт»']
[' Уважаемый Эдвард Станиславович!Примите поздравления с\xa085-летием.Человек многогранного дарования и\xa0большой созидательной энергии, Вы по\xa0праву входите в\xa0число признанныхдраматургов, мастеров современного театрального искусства. И\xa0конечно, Васхорошо знают и\xa0любят как автора целого ряда просветительских проектов, чьётворчество завоевало сердца самой широкой аудитории, в\xa0том числе молодёжи.Желаю доброго здоровья, благополучия и\xa0бодростидуха. Владимир Путин     Статус материала Опубликован в\xa0разделе: Телеграммы  Ссылка на материал: kremlin.ru/d/66751   Текстовая версия      ', ' Дорогие друзья!Поздравляю вас со\xa0знаменательным юбилеем\xa0– 30-летием радиостанции «Орфей».Все эти годы ваш коллектив объединяет настоящих единомышленников, профессионалов, преданных высокому искусству, стремящихся внести свой вкла

In [15]:
data = {'Title': titles_list,
        'Content': congrats_list}

my_df = pd.DataFrame(data)
my_df

,Title,Content
0,"Эдварду Радзинскому, писателю, драматургу, сце...",Уважаемый Эдвард Станиславович!Примите поздра...
1,Коллективу радиостанции «Орфей»,Дорогие друзья!Поздравляю вас со знаменательн...
2,"Михаилу Ковальчуку, президенту НИЦ «Курчатовск...",Уважаемый Михаил Валентинович!Примите мои поз...
3,"Ирине Андреевой и Ивану Штылю, победителям чем...",Уважаемые Ирина Александровна и Иван Александ...
4,Работникам и ветеранам оборонно-промышленного ...,Уважаемые друзья!Поздравляю вас с Днём оружей...
...,...,...
883,Жителям Республики Мордовия,Дорогие друзья!Поздравляю вас с юбилеем – 75-...
884,Коллективу Института физических проблем им.П.Л...,Поздравляю коллектив института с 70-летием со...
885,Жителям Красноярского края,Уважаемые друзья!Поздравляю вас с 70-летием с...
886,В.А.ЛОГИНОВУ,Уважаемый Владимир Александрович!Поздравляю В...


In [16]:
! pip install datasets transformers

In [17]:
!git clone  https://github.com/sberbank-ai/ru-gpts

fatal: destination path 'ru-gpts' already exists and is not an empty directory.


In [18]:
from datasets import load_dataset
from sklearn.model_selection import train_test_split
import re
# datasets = load_dataset("text", data_files={"train": path_to_train.txt, "validation": path_to_validation.txt}

In [19]:
my_df['Sum'] = [my_df['Title'].tolist()[i] + my_df['Content'].tolist()[i] for i in range(len(my_df))]
my_df

,Title,Content,Sum
0,"Эдварду Радзинскому, писателю, драматургу, сце...",Уважаемый Эдвард Станиславович!Примите поздра...,"Эдварду Радзинскому, писателю, драматургу, сце..."
1,Коллективу радиостанции «Орфей»,Дорогие друзья!Поздравляю вас со знаменательн...,Коллективу радиостанции «Орфей» Дорогие друзья...
2,"Михаилу Ковальчуку, президенту НИЦ «Курчатовск...",Уважаемый Михаил Валентинович!Примите мои поз...,"Михаилу Ковальчуку, президенту НИЦ «Курчатовск..."
3,"Ирине Андреевой и Ивану Штылю, победителям чем...",Уважаемые Ирина Александровна и Иван Александ...,"Ирине Андреевой и Ивану Штылю, победителям чем..."
4,Работникам и ветеранам оборонно-промышленного ...,Уважаемые друзья!Поздравляю вас с Днём оружей...,Работникам и ветеранам оборонно-промышленного ...
...,...,...,...
883,Жителям Республики Мордовия,Дорогие друзья!Поздравляю вас с юбилеем – 75-...,Жителям Республики Мордовия Дорогие друзья!Поз...
884,Коллективу Института физических проблем им.П.Л...,Поздравляю коллектив института с 70-летием со...,Коллективу Института физических проблем им.П.Л...
885,Жителям Красноярского края,Уважаемые друзья!Поздравляю вас с 70-летием с...,Жителям Красноярского края Уважаемые друзья!По...
886,В.А.ЛОГИНОВУ,Уважаемый Владимир Александрович!Поздравляю В...,В.А.ЛОГИНОВУ Уважаемый Владимир Александрович!...


In [20]:
train_test_ratio = 0.9
df_train, df_test = train_test_split(my_df, train_size = train_test_ratio, random_state = 1)
# train_valid_ratio = 7/9
# df_train, df_valid = train_test_split(df_full_train, train_size = train_valid_ratio, random_state = 1)

In [21]:
def build_dataset(df, dest_path):
    f = open(dest_path, 'w')
    data = ''
    summaries = df['Sum'].tolist()
    # c = 0
    # if c < 2:
    #   print(summaries)
    #   c += 1
    for summary in summaries:
        summary = str(summary).strip()
        summary = re.sub(r"\s", " ", summary)
        summary = re.sub(r"\n", "", summary)
        summary = re.sub(r"Статус материала Опубликован в разделе: Телеграммы", "", summary)
        summary = re.sub(r"Ссылка на материал:", "", summary)
        summary = re.sub(r"Ссылка на материал:", "", summary)
        summary = re.sub(r"([a-zA-Z]+)", "", summary)
        summary = re.sub(r"([0-9/*#@+]+)", "", summary)
        # bos_token = '<BOS>'
        # eos_token = '<EOS>'
        # data += bos_token + ' ' + summary + ' ' + eos_token + '\n'
        data += summary
    # c = 0
    # if c < 2:
    #   print(data)
    #   c += 1
    f.write(data)

In [22]:
!ls
!rm 'train.txt' 'test.txt'
!ls

cached_lm_GPT2Tokenizer_128_test.txt	    curr_congrats.txt.2  ru-gpts
cached_lm_GPT2Tokenizer_128_test.txt.lock   curr_congrats.txt.3  sample_data
cached_lm_GPT2Tokenizer_128_train.txt	    curr_congrats.txt.4  test.txt
cached_lm_GPT2Tokenizer_128_train.txt.lock  curr_congrats.txt.5  train.txt
curr_congrats.txt			    gpt2		 transformers
curr_congrats.txt.1			    gpt-letters		 valid.txt
cached_lm_GPT2Tokenizer_128_test.txt	    curr_congrats.txt.2  ru-gpts
cached_lm_GPT2Tokenizer_128_test.txt.lock   curr_congrats.txt.3  sample_data
cached_lm_GPT2Tokenizer_128_train.txt	    curr_congrats.txt.4  transformers
cached_lm_GPT2Tokenizer_128_train.txt.lock  curr_congrats.txt.5  valid.txt
curr_congrats.txt			    gpt2
curr_congrats.txt.1			    gpt-letters


In [23]:
build_dataset(df_train, 'train.txt')
build_dataset(df_test, 'test.txt')
# build_dataset(df_valid, 'valid.txt')

In [24]:


# tokenizer = AutoTokenizer.from_pretrained("anonymous-german-nlp/german-gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")
 
train_path = 'train.txt'
test_path = 'test.txt'

In [25]:
!rm ./gpt-letters

rm: cannot remove './gpt-letters': Is a directory


In [26]:
# tokenizer.add_special_tokens({'bos_token': '<BOS>', 'eos_token': '<EOS>', 'mask_token': '<MASK>'})

3

In [27]:
from transformers import TextDataset, DataCollatorForLanguageModeling

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset, test_dataset, data_collator

train_dataset, test_dataset, data_collator = load_dataset(train_path, test_path, tokenizer)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


## **TRAIN MODEL**

In [28]:
from transformers import Trainer, TrainingArguments, AutoModelWithLMHead

model = GPT2LMHeadModel.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")


training_args = TrainingArguments(
    output_dir="./gpt-letters", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    # num_train_epochs=3, # number of training epochs
    # per_device_train_batch_size=32, # batch size for training
    # per_device_eval_batch_size=64,  # batch size for evaluation
    num_train_epochs=2, # number of training epochs
    per_device_train_batch_size=24, # batch size for training
    per_device_eval_batch_size=32,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=150, # after # steps model is saved 
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [29]:
trainer.train()

***** Running training *****
  Num examples = 1269
  Num Epochs = 2
  Instantaneous batch size per device = 24
  Total train batch size (w. parallel, distributed & accumulation) = 24
  Gradient Accumulation steps = 1
  Total optimization steps = 106


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=106, training_loss=2.5961516758181014, metrics={'train_runtime': 4833.9866, 'train_samples_per_second': 0.525, 'train_steps_per_second': 0.022, 'total_flos': 165789794304000.0, 'train_loss': 2.5961516758181014, 'epoch': 2.0})

In [30]:
trainer.save_model()

Saving model checkpoint to ./gpt-letters
Configuration saved in ./gpt-letters/config.json
Model weights saved in ./gpt-letters/pytorch_model.bin


In [54]:
# my_model = torch.load('./gpt-letters/pytorch_model.bin')
from transformers import pipeline

pu = pipeline('text-generation',model='./gpt-letters', tokenizer=tokenizer, max_length=500)

loading configuration file ./gpt-letters/config.json
Model config GPT2Config {
  "_name_or_path": "./gpt-letters",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 2048,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 2048,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "torch_dtype": "float32",
  "transformers_version": "4.14.1",
  "use_cache": true,
  "vocab_size": 50264
}

loading configuration file ./gpt-letters/config.json
Model config GPT2Config {
  "_

In [55]:
pu("Александру Сергеевичу Пушкину")[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Александру Сергеевичу Пушкину, поэту, драматургу, лауреату Государственной премии СССР, лауреату Государственной премии Российской Федерации в области литературы и искусства, заслуженному деятелю искусств России, лауреату Государственной премии Российской Федерации в области литературы и искусства, заслуженному деятелю искусств России, лауреату Государственной премии Российской Федерации в области литературы и искусства, заслуженному деятелю искусств России, лауреату Государственной премии Российской Федерации в области литературы и искусства, заслуженному деятелю искусств России, лауреату Государственной премии Российской Федерации в области литературы и искусства, заслуженному деятелю искусств России, лауреату Государственной премии Российской Федерации в области литературы и искусства, заслуженному деятелю искусств России, лауреату Государственной премии Российской Федерации в области литературы и искусства, заслуженному деятелю искусств России, лауреату Государственной премии Росс

In [56]:
pu("Кровавым убийцам детей")[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Кровавым убийцам детей и внуков.Желаю вам здоровья, благополучия и всего самого доброго. Владимир Путин     Статус материала Опубликован в разделе: Телеграммы  Ссылка на материал: kremlin.ru/d/6166   Текстовая версия <EOS>\n<BOS> Уважаемый Владимир Владимирович!Поздравляю Вас с Днём рождения.Вы – яркий, самобытный, самобытный учёный, талантливый педагог, талантливый организатор, талантливый организатор. Вы – яркий, самобытный, самобытный, самобытный учёный, талантливый организатор. Вы – яркий, самобытный, самобытный, самобытный, самобытный, самобытный, самобытный, самобытный, самобытный, самобытный, самобытный, самобытный, самобытный, самобытный, самобытный, самобытный, самобытный, самобытный, самобытный, самобытный, самобытный, самобытный, самобытный, самобытный, самобытный, самобытный, самобытный, самобытный, самобытный, самобытный, самобытный, самобытный, самобытный, самобытный, самобытный, самобытный, самобытный, самобытный, самобытный, самобытный, самобытный, самобытный, самобытн

In [57]:
pu("Уважаемые осквернители святынь и насильники")[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Уважаемые осквернители святынь и насильники, выносите свой протест против осквернения святынь и осквернения храмов.Желаю вам здоровья, благополучия и всего самого доброго. Владимир Путин     Статус материала Опубликован в разделе: Телеграммы  Ссылка на материал: kremlin.ru/d/6166   Текстовая версия <EOS>\n<BOS> Уважаемый Владимир Владимирович!Поздравляю вас с Днём рождения.Вы – яркий, самобытный, самобытный учёный, талантливый учёный, педагог, автор многих научных трудов. Вы – яркий, самобытный, самобытный учёный, педагог, автор многих научных трудов.Желаю вам здоровья, благополучия и всего самого доброго. Владимир Путин     Статус материала Опубликован в разделе: Телеграммы  Ссылка на материал: kremlin.ru/d/6166   Текстовая версия <EOS>\n<BOS> Уважаемый Владимир Владимирович!Поздравляю вас с Днём рождения.Вы – яркий, самобытный, самобытный, самобытный учёный, педагог, автор многих научных трудов.Желаю вам здоровья, благополучия и всего самого доброго. Владимир Путин     Статус матери

In [58]:
pu("Хочу сказать этому Обэме")[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Хочу сказать этому Обэме, что он не только не утратил своей былой славы, но и продолжает оставаться одним из самых ярких и самобытных представителей своего поколения. Желаю Вам новых успехов и всего самого доброго. Владимир Путин     Статус материала Опубликован в разделе: Телеграммы  Ссылка на материал: kremlin.ru/d/6166   Текстовая версия <EOS>\n<BOS> Уважаемый Владимир Владимирович! Поздравляю Вас с Днём рождения. Вы – яркий, самобытный, самобытный человек, обладаете неиссякаемой энергией, энергией, энергией, энергией, энергией, энергией, энергией, энергией, энергией, энергией, энергией, энергией, энергией, энергией, энергией, энергией, энергией, энергией, энергией, энергией, энергией, энергией, энергией, энергией, энергией, энергией, энергией, энергией, энергией, энергией, энергией, энергией, энергией, энергией, энергией, энергией, энергией, энергией, энергией, энергией, энергией, энергией, энергией, энергией, энергией, энергией, энергией, энергией, энергией, энергией, энергией, э

In [59]:
pu("Куцеву Диме")[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Куцеву Диме, тренеру сборной России по хоккею с мячом, победителю XXII Олимпийских зимних игр в Сочи в соревнованиях по биатлону в дисциплине спринт на дистанции 100 метров.Желаю вам новых успехов и всего самого доброго. Владимир Путин     Статус материала Опубликован в разделе: Телеграммы  Ссылка на материал: kremlin.ru/d/6166   Текстовая версия <EOS>\n<BOS> Уважаемый Владимир Владимирович!Поздравляю Вас с Днём рождения.Вы – яркий, самобытный, самобытный человек, обладаете неиссякаемой энергией, энергией, целеустремлённостью, целеустремлённостью.Желаю Вам здоровья, благополучия и всего самого доброго. Владимир Путин     Статус материала Опубликован в разделе: Телеграммы  Ссылка на материал: kremlin.ru/d/6166   Текстовая версия <EOS>\n<BOS> Уважаемый Владимир Владимирович!Поздравляю Вас с Днём рождения.Вы – яркий, самобытный, самобытный, самобытный, самобытный, самобытный, самобытный, самобытный, самобытный, самобытный, самобытный, самобытный, самобытный, самобытный, самобытный, самоб

In [60]:
pu("Мычалкин")[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Мычалкин, Владимир Владимирович, Вы – один из самых ярких, талантливых, талантливых, востребованных специалистов в области медицины. Уверен, что Вы и впредь будете востребованы в нашей стране. Желаю Вам новых успехов и всего самого доброго. Дмитрий Медведев     Статус материала Опубликован в разделе: Телеграммы  Ссылка на материал: kremlin.ru/d/6166   Текстовая версия <EOS>\n<BOS> Уважаемый Владимир Владимирович! Поздравляю Вас с Днём рождения. Вы – один из самых ярких, талантливых, востребованных специалистов в области медицины. Уверен, что Вы и впредь будете востребованы в нашей стране. Желаю Вам новых успехов и всего самого доброго. Дмитрий Медведев     Статус материала Опубликован в разделе: Телеграммы  Ссылка на материал: kremlin.ru/d/6166   Текстовая версия <EOS>\n<BOS> Уважаемый Владимир Владимирович! Поздравляю Вас с Днём рождения. Вы – один из самых ярких, талантливых, востребованных специалистов в области медицины. Уверен, что Вы и впредь будете востребованы в нашей стране. 

In [36]:
from google.colab import files

In [37]:
import os
for file in os.listdir('./gpt-letters'):
  files.download(f'./gpt-letters/{file}')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [38]:
import os
for file in os.listdir('./gpt-letters'):
  print(file)

config.json
pytorch_model.bin
training_args.bin
runs
